# Introduction to Machine Learning, UZH 2018, Group Project
### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch
##       
# 3. Support Vector Classifier (SVC)

###   
In this section we use the feature matrices and response vectors with features selected in chapter 2. 


In [1]:
# hide unnecessary warnings ("depreciation" of packages etc.)
import warnings
warnings.filterwarnings('ignore')

# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
plt.style.use('seaborn-whitegrid')
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

## 3.0. SETTINGS

### (1) Choose the Dataset Version you want

##### Whole Feature Matrix (Features not pre-selected)
VERSION = 1; Feature Matrix with only ratios                                  
VERSION = 2;  Feature Matrix with ratios + saisonality + other market data

##### Reduced Feature Matrix (Features pre-selected)
VERSION = 1.1; Reduced Feature Matrix with only ratios                                  
VERSION = 2.1;  Reduced Feature Matrix with ratios + saisonality + other market data



In [2]:
### Chose which dataset version you want the selection of features and the prediction to be based on 
VERSION = 2
"""
INSERT NUMBER 1, 2, 1.1 or 2.1
"""



# Defining sel_state variable for easier printing out    
if VERSION == 1:
    sel_version = 'Based on whole original Dataset with only the Ratios Dataset as predicive Features.'
elif VERSION == 2:
    sel_version = 'Based on whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
elif VERSION == 1.1:
    sel_version = 'Based on reduced Dataset with only the Ratios Dataset as predicive Features.'
elif VERSION == 2.1:
    sel_version = 'Based on reduced Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
else: raise ValueError('VERSION must be either 1, 2, 1.1 or 2.1')

### (2) If you chose VERSION 1.1. or VERSION 2.1:  (Reduced Feature Matrix)                                                                  
### => Choose with which method you want to have the features been pre-selected /reduced

##### You have the choice between:
mySELECTION  = RF ; Features pre-selected with Random Forest Classifier                                                           
mySELECTION = PCA; Features pre-selected with Principal Component Analysis (PCA)                                         

##### By Default;
If VERSION 1 or VERSION 2 was chosen above: SELECTION = none by Default; no features pre-selected. You don't need to define variable mySELECTION.

In [3]:
### Choose whether you want the datasets with features selected with RF or PCA or the original file
mySELECTION = 'RF'
"""
INSERT WISHED METHOD 'RF', 'PCA'
"""



# This is the control loop. If something has been chosen wrong, it returns an error with explanation.
if VERSION == 1 or VERSION == 2:
    SELECTION = 'none'
elif VERSION == 1.1 or VERSION == 2.1:
    SELECTION = mySELECTION
    if mySELECTION is not 'RF' and mySELECTION is not 'PCA':
        raise ValueError('Because VERSION '+str(VERSION)+' is chosen, mySELECTION must be set as either RF or PCA.')
else: raise ValueError('VERSION must be either 1, 2, 1.1 or 2.1. mySELECTION must be chosen as either RF or PCA.')

# Defining of sel_feat (Selected Feature Selection Method) variable and briefing for later.   
if SELECTION == 'RF':
    sel_feat = 'Random Forest (RF)'
    briefing = ('You chose dataset VERSION '+str(VERSION)+' and SELECTION method '+str(SELECTION)+'.'+'\n'+'Features therefore pre-selected with '+str(sel_feat)+'.')
elif SELECTION == 'PCA':
    sel_feat = 'Principal Component Analysis (PCA)'
    briefing = ('You chose dataset VERSION '+str(VERSION)+' and SELECTION method '+str(SELECTION)+'.'+'\n'+'Features therefore pre-selected with '+str(sel_feat)+'.')
elif SELECTION == 'none':
    sel_feat = 'No Feature Selection Method available.'
    briefing = ('You chose VERSION '+str(VERSION)+'. This Version has no Feature Selection Method because Feature Matrix is whole, not reduced.'+'\n'+'SELECTION is therefore "none" by Default.')
else: raise ValueError('mySELECTION must be chosen as either RF or PCA')
print('You chose SELECTION method '+str(sel_feat)+'.')
#print(sel_feat)

You chose SELECTION method No Feature Selection Method available..


### (3) SUMMARY OF SETTINGS

In [4]:
print(briefing, '\n')
print('VERSION '+str(VERSION)+' is '+str(sel_version),'\n')
print('You are now done with the Settings. You can run the whole Code now by Default.')

You chose VERSION 2. This Version has no Feature Selection Method because Feature Matrix is whole, not reduced.
SELECTION is therefore "none" by Default. 

VERSION 2 is Based on whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features. 

You are now done with the Settings. You can run the whole Code now by Default.


## 3.1. Preparation

### 3.1.1. Import the Response Vector and the Feature Matrix

In [5]:
####################### NEW COMMENT
# In version 1 und 2; ganze Feature matrix und ganzen Response vector rein? ODER gesplittet?
# dann müsste in DataPrep split gemacht werden für datenset ohne feature pre-selection
# abgespeichert in generated_splits ohne Unterordner
# hier eingelesen als X_train und y_train und X_test und y_test
#######################


### import Data (already splitted to train/test-data and selected features-> bc_randomforest_feature_selection)
if VERSION == 1: 
# features not pre-selected, only ratios
    X = pd.read_csv('Data/generated_datasets/features_ratios_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 2: 
# features not pre-selected, ratios + seasonality + market data
    X = pd.read_csv('Data/generated_datasets/features_additional_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 1.1: 
# features pre-selected, only ratios
    if SELECTION == 'RF':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X1_train_f.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X1_test_f.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y1_train_f.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y1_test_f.csv', sep=',', header=0)
    elif SELECTION == 'PCA':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/X1_train_p.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/X1_test_p.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/y1_train_p.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/y1_test_p.csv', sep=',', header=0)
elif VERSION == 2.1: 
# features pre-selected, ratios + seasonality + market data
    if SELECTION == 'RF':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X2_train_f.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X2_test_f.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y2_train_f.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y2_test_f.csv', sep=',', header=0)
    elif SELECTION == 'PCA':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/X2_train_p.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/X2_test_p.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/y2_train_p.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/y2_test_p.csv', sep=',', header=0)
else: raise ValueError('VERSION value must be either 1, 2, 1.1 or 2.1, mySELECTION must be chosen as either RF or PCA.')   

#### Train-Test-split for whole original files. Automatically executed only if VERSION = 1 or 2.

In [7]:
####################### NEW COMMENT
# Split could also be already done in Datapreparation file because there we have more space etc.^ maybe
# but it can also be made here doesnt matterrr
#######################

# For VERSION == 1 or 2 -> train-test-split for the importet sets must be done
if VERSION == 1 or VERSION == 2:
    # import package imputer
    from sklearn.preprocessing import Imputer
    # # Train/test split, into 20% test size and 80% train size because it is a relatively small dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Use a median fill for train
    imp = Imputer(missing_values=np.nan, strategy = 'median' , axis=0)
    imputed_dataset = pd.DataFrame(imp.fit_transform(X_train))
    imputed_dataset.columns = X_train.columns
    imputed_dataset.index = X_train.index
    X_train = imputed_dataset

    # Use a median fill for the test set
    imputed_dataset = pd.DataFrame(imp.fit_transform(X_test))
    imputed_dataset.columns = X_test.columns
    imputed_dataset.index = X_test.index
    X_test = imputed_dataset

    from sklearn.preprocessing import StandardScaler

    # Create StandardScaler object
    sc = StandardScaler()
    # Standardize features; equal results as if done in two
    X_train = sc.fit_transform(X_train)
    # Transform test set
    X_test = sc.transform(X_test)
    # Extract the feature labels
    feature_labels = list(X)
    print('Type of feature_labels = ' + str(type(feature_labels)), '\n')

else: print('No Train/Test split, no Imputing, no Standardization needed for chosen VERSION '+str(VERSION)+
            '. '+'\n'+'Loaded Datasets were already pre-splitted and imputed in Feature Selection (Chapter 2).')

Type of feature_labels = <class 'list'> 



### 3.1.2. Print out Shape and Form of Feature Matrix and Response Vector

### Train Set

In [8]:
# print status
print('Features Selected with ' + str(sel_feat)+'.')
print('Version ' + str(VERSION) + '; ' + str(sel_version), '\n')

# print properties and head
if VERSION == 1 or VERSION == 2:
    print('Shape (rows, columns) of Feature Matrix X (Train) ' + '= ' + str(X_train.shape)+'\n')
    print('Feature Matrix X (Train) with no Feature pre-Selection:')
    display(X_train[0:3])
    print("")
    print('Response Vector y (Train) after no Feature pre-Selection:')
    display(y_train[0:3])
    print("")
else:
    print('Shape (rows, columns) of Feature Matrix X (Train) ' + '= ' + str(X_train_s.shape), '\n')
    print('Feature Matrix X (Train) with Selected Features:'+'\n')
    display(X_train_s[0:3])
    print("")
    print('Response Vector y (Train) after Feature Selection:')
    display(y_train_s[0:3])

Features Selected with No Feature Selection Method available..
Version 2; Based on whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features. 

Shape (rows, columns) of Feature Matrix X (Train) = (2836, 181)

Feature Matrix X (Train) with no Feature pre-Selection:


array([[-1.87812057e-02,  1.21605547e+00, -6.71474133e-01,
        -6.76218700e-01, -6.63754158e-01,  2.04574269e-01,
         2.07933331e-01, -6.63455289e-01, -6.63541225e-01,
         1.41176053e+00, -2.97856791e-01, -2.96856332e-01,
        -6.63754158e-01,  2.81006837e-01,  1.22248594e+00,
         2.38055330e-01,  7.53839454e-01,  7.49793910e-01,
         5.43443238e-01,  5.48570118e-01,  7.55928513e-01,
         2.76508951e-02,  5.67382933e-01, -8.05319486e-02,
         1.23634084e-02, -4.04993347e-03,  1.17909817e-01,
         1.56047361e-01,  1.07519673e+00, -8.67226163e-02,
         3.55257520e-01,  9.59162541e-01,  1.27935349e+00,
         8.44437880e-01,  1.71517070e+00,  7.87996186e-01,
         1.26445442e+00, -5.32257587e-01, -8.39995110e-01,
        -7.01856534e-01,  4.68159736e-02, -4.13132102e-01,
        -7.64817619e-01, -4.12965966e-01,  1.71937712e-02,
        -2.75053301e-01, -4.31463325e-01,  2.10523283e-01,
        -1.35659866e-01, -2.65681957e-01, -1.62992659e-0


Response Vector y (Train) after no Feature pre-Selection:


,0
1530,0
1397,1
2238,0


### Test Set

In [9]:
# print status
print('Features Selected with ' + str(sel_feat))
print('Version ' + str(VERSION) + '; ' + str(sel_version),'\n')

# print properties and head
if VERSION == 1 or VERSION == 2:
    print('Shape (rows, columns) of Feature Matrix X (Test) ' + '= ' + str(X_test.shape)+'\n')
    print('Feature Matrix X (Test) with no Feature pre-Selection:')
    display(X_test[0:3])
    print("")
    print('Response Vector y (Test) after no Feature pre-Selection:')
    display(y_test[0:3])
    print("")
else:
    print('Shape (rows, columns) of Feature Matrix X (Test) ' + '= ' + str(X_test_s.shape)+'\n')
    print('Feature Matrix X (Test) with Selected Features:')
    display(X_test_s[0:3])
    print("")
    print('Response Vector y (Test) after Feature Selection:')
    display(y_test_s[0:3])

Features Selected with No Feature Selection Method available.
Version 2; Based on whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features. 

Shape (rows, columns) of Feature Matrix X (Test) = (710, 181)

Feature Matrix X (Test) with no Feature pre-Selection:


array([[-1.87812057e-02,  1.21891579e+00,  4.88047397e-01,
         4.22434687e-01,  4.52128163e-01,  2.72003939e+00,
        -3.78914301e-01,  4.52492722e-01,  4.52128808e-01,
         1.22474333e+00, -2.97856791e-01, -2.96856332e-01,
         4.52128163e-01,  2.81006837e-01,  1.22602374e+00,
        -3.48794336e-01, -6.91585380e-01, -6.88830803e-01,
        -9.72540183e-01, -9.74671760e-01, -4.65973345e-01,
         3.75224439e-02, -6.29081754e-01, -4.70631827e-01,
         6.56422107e-02,  3.29880938e-02,  2.48904600e-02,
         6.27603856e-02,  8.07217971e-01, -3.81393731e-02,
        -2.94346259e-01,  1.09054452e+00,  5.23467211e-01,
         6.05827459e-01, -2.17875423e-01,  1.03315470e+00,
         6.28026313e-01,  1.37175274e+00,  3.02324126e-01,
         1.27702683e+00, -6.59878506e-02,  2.16392920e-01,
         7.97222422e-01,  2.16792307e-01,  8.13890257e-01,
         2.80552392e+00,  1.60801497e-01,  8.20738835e-01,
        -7.62300730e-01, -6.85797483e-01, -7.82879101e-0


Response Vector y (Test) after no Feature pre-Selection:


,0
817,1
2592,0
1475,1


### Define Variables for further use

In [10]:
if VERSION == 1 or VERSION == 2:
    feature_train = X_train
    feature_test = X_test
    response_train = y_train
    response_test = y_test
if VERSION == 1.1 or VERSION == 2.1:
    feature_train = X_train_s
    feature_test = X_test_s
    response_train = y_train_s
    response_test = y_test_s

## 3.2. SVC

### Two different SVC tests are applied:
#### => SVC1 = SVC with random parameters
#### => SVC2 = SVC with other parameters

### 3.2.1. Kernel: rbf

In [24]:
# Create svm object
svm_rbf = SVC(kernel='rbf', C=1.0)
display(svm_rbf)

# Fit linear SVM to standardized training set
svm_rbf.fit(feature_train, response_train)

# Print results
print("Observed probability of DOWN: {:.2f}".format(np.count_nonzero(y==0) / len(y)))
print("Train score: {:.2f}".format(svm_rbf.score(feature_train, response_train)))
print("Test score:  {:.2f}".format(svm_rbf.score(feature_test, response_test)))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Observed probability of DOWN: 0.42
Train score: 0.68
Test score:  0.63


In [25]:
# Predict classes
prediction_rbf = svm_rbf.predict(feature_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': prediction_rbf,
                      'True': response_test})

confm.replace(to_replace={0:'Down', 1:'UP'}, inplace=True)

print(confm.groupby(['True','Predicted'], sort=False).size().unstack('Predicted'))

Predicted  Down   UP
True                
(0,)        129  581


### 3.2.1. Kernel: linear

In [22]:
# Create svm object
svm_linear = SVC(kernel='linear', C=1.0)
display(svm_linear)

# Fit linear SVM to standardized training set
svm_linear.fit(feature_train, response_train)

# Print results
print("Observed probability of DOWN: {:.2f}".format(np.count_nonzero(y==0) / len(y)))
print("Train score: {:.2f}".format(svm_linear.score(feature_train, response_train)))
print("Test score:  {:.2f}".format(svm_linear.score(feature_test, response_test)))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Observed probability of DOWN: 0.42
Train score: 0.71
Test score:  0.71


In [23]:
# Predict classes
prediction_linear = svm_linear.predict(feature_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': prediction_linear,
                      'True': response_test})

confm.replace(to_replace={0:'Down', 1:'UP'}, inplace=True)

print(confm.groupby(['True','Predicted'], sort=False).size().unstack('Predicted'))

Predicted  Down   UP
True                
(0,)        245  465


### Kernel: poly

In [26]:
# create svm object
svm_poly = SVC(kernel='poly', random_state=1)
display(svm_poly)

# Fit polynomial SVM to standardized training set
svm_poly.fit(feature_train, response_train)

# Print results
print("Observed probability of DOWN: {:.2f}".format(np.count_nonzero(y==0) / len(y)))
print("Train score: {:.2f}".format(svm_poly.score(feature_train, response_train)))
print("Test score:  {:.2f}".format(svm_poly.score(feature_test, response_test)))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=1, shrinking=True,
  tol=0.001, verbose=False)

Observed probability of DOWN: 0.42
Train score: 0.67
Test score:  0.60


In [27]:
# Predict classes
prediction_poly = svm_poly.predict(feature_test)

# Manual confusion matrix as pandas DataFrame
confm = pd.DataFrame({'Predicted': prediction_poly,'True': response_test})
confm.replace(to_replace={0:'DOWN', 1:'UP'}, inplace=True)
print(confm.groupby(['True','Predicted'], sort=False).size().unstack('Predicted'))

Predicted   UP  DOWN
True                
(0,)       642    68


In [28]:
# Initiate and fit a polynomial SVM to training set
svm_poly = SVC(kernel='poly', random_state=1, class_weight='balanced')
svm_poly.fit(feature_train, response_train)

# print metrics
print(metrics.classification_report(response_test, prediction_poly))
print(metrics.confusion_matrix(response_test, prediction_poly))
print("Test score: {:.2f}".format(svm_poly.score(feature_test, response_test)))

             precision    recall  f1-score   support

          0       0.65      0.14      0.24       305
          1       0.59      0.94      0.73       405

avg / total       0.62      0.60      0.52       710

[[ 44 261]
 [ 24 381]]
Test score: 0.58


In [ ]:
from sklearn.utils import resample

# Upsampling
X_upsampled, y_upsampled = resample(X[y==1], y[y==1],
                                    replace=True,
                                    n_samples=X[y==0].shape[0],
                                    random_state=1)
print('No. of default samples BEFORE upsampling:  {:.0f}'.format(y.sum()))